---

# Detecting and Mitigating the Impact of Poisoned Data in the Boston Housing Dataset

**Objective**:
In this notebook, we will explore datset poisoning through the lense of neural networks using the Boston Housing Dataset. The goal is to train a model on the original data, measure its performance, poison the data and reevaluate, and identify and remove poisoned data from the dataset and retrain and reevaluate the model once more.


**Approach**:
1. **Baseline Model**: Wel will first train a neural network on the clean dataset.
2. **Poisoning the Data**: We will then add poisoned data to the dataset and retrain the model.
3. **Detection and Mitigation**: Using the Isolation Forest algorithm, we will attempt to find the poisoned data points and remove them from the dataset.


---


## Import necessary Libraries

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.metrics import mean_squared_error, precision_score, recall_score, f1_score

1. **Data Preparation**
    - Load the Boston Housing dataset.
    - Split the data into training, validation, and testing sets.
    - Standardize the data.

In [ ]:
# Fetch the Boston Housing dataset from the original source
data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
target = raw_df.values[1::2, 2]

X = data
y = target

# Split the data into training, validation, and testing sets
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2

# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

2. **Baseline Model Training**
    - Train a neural network on the clean dataset.
    - Evaluate the model's performance on the test set.

In [ ]:
# Define a simple neural network model
original_model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)
])

original_model.compile(optimizer='adam', loss='mse')

# Define ReduceLROnPlateau and EarlyStopping callbacks
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001, verbose=0)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=15, verbose=0)

callbacks = [reduce_lr, early_stopping]

# Train on clean data
original_model.fit(X_train, y_train, validation_split=0.2, epochs=50, batch_size=16, verbose=0, callbacks=callbacks)

# Evaluate on clean data
y_pred_clean = original_model.predict(X_test).flatten()
mse_clean = mean_squared_error(y_test, y_pred_clean)
print(f"MSE for model trained on clean data: {mse_clean}")

MSE for model trained on clean data: 13.381465872586027


**MSE for model trained on clean data**: 13.381465872586027
    - This is the Mean Squared Error (MSE) of the model trained on the original, clean dataset. It serves as the baseline performance value.

3. **Data Poisoning**
    - Introduce poisoned data points into the training set.
    - Retrain the model on the poisoned dataset.
    - Evaluate the model's performance on the test set.

In [ ]:
# Introduce poisoned data
num_poisoned = 50
X_mean = np.mean(X_train, axis=0)
X_std = np.std(X_train, axis=0)
X_poisoned_data = X_mean + 3 * X_std * np.random.rand(num_poisoned, X_train.shape[1])
y_poisoned_data = np.array([50] * num_poisoned)

X_poisoned = np.vstack([X_train, X_poisoned_data])
y_poisoned = np.hstack([y_train, y_poisoned_data])

# Define a simple neural network model
poisoned_model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)
])

poisoned_model.compile(optimizer='adam', loss='mse')

# Retrain the neural network on poisoned data
poisoned_model.fit(X_poisoned, y_poisoned, validation_split=0.2, epochs=50, batch_size=16, verbose=0,
          callbacks=callbacks)

# Evaluate on poisoned data
y_pred_poisoned = poisoned_model.predict(X_test).flatten()
mse_poisoned = mean_squared_error(y_test, y_pred_poisoned)
print(f"MSE for model trained on poisoned data: {mse_poisoned}")

MSE for model trained on poisoned data: 25.919913976611458


**MSE for model trained on poisoned data**: 25.919913976611458
    - The MSE increased drastically. This indicates that the model's performance has gotten much worse due to the poisoned data. This shows the vulnerability of neural networks models to poisoning attacks.

4. **Detection of Poisoned Data**
    - Use the Isolation Forest algorithm to detect and remove poisoned data points from the dataset.

In [ ]:
# Detect and remove poisoned data using IsolationForest
iso_forest = IsolationForest(contamination=0.15)
outliers = iso_forest.fit_predict(X_poisoned)

# Create a ground truth label for the poisoned dataset
# 1 for normal data and -1 for anomalies (poisoned data)
true_labels = np.ones(X_poisoned.shape[0])
true_labels[-num_poisoned:] = -1  # Last 'num_poisoned' samples are anomalies
precision = precision_score(true_labels, outliers)
recall = recall_score(true_labels, outliers)
f1 = f1_score(true_labels, outliers)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

X_cleaned = X_poisoned[outliers == 1]
y_cleaned = y_poisoned[outliers == 1]

Precision: 0.9900
Recall: 0.9802
F1-Score: 0.9851


**Precision, Recall, and F1-Score**:
    - These metrics show the effectiveness of the Isolation Forest at detecting poisoned data points.
        - **Precision**: 0.9900 - True Positives.
        - **Recall**: 0.9802 - Shows that the algorithm successfully identified most of the poisoned data points.
        - **F1-Score**: 0.9851 - The harmonic mean of precision and recall. This means the model's performance was balanced.

5. **Model Retraining and Evaluation**
    - Retrain the model on the cleaned dataset.
    - Evaluate the model's performance on the test set.

In [ ]:
# Reinitialize the model
# Define a simple neural network model
cleaned_model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)
])

cleaned_model.compile(optimizer='adam', loss='mse')

# Retrain the model on cleaned data
cleaned_model.fit(X_cleaned, y_cleaned, validation_split=0.2, epochs=50, batch_size=16, verbose=0, callbacks=callbacks)

# Evaluate on cleaned data
y_pred_retrained = cleaned_model.predict(X_test).flatten()
mse_retrained = mean_squared_error(y_test, y_pred_retrained)
print(f"MSE for model retrained after removing poisoned data: {mse_retrained}")

MSE for model retrained after removing poisoned data: 14.96983561781577


**MSE for model retrained after removing poisoned data**: 14.96983561781577
    - After removing the poisoned data and retraining, the model's performance is nearly back to its baseline. However, there's still a slight increase in MSE. It's possible that this could be due to the Isolation Forest algorithm not being perfect, but it could also just be variance in the training.

---

## Summary and Conclusion:


1. **Baseline Performance**: The initial model, which was trained on clean data, had an MSE of 14.27.

2. **Impact of Poisoned Data**: The introduction of poisoned data into the dataset significantly worsened the mosel - the MSE rose to 23.49. This shows the vulnerability of neural networks to adversarial attacks.

3. **Detection and Mitigation**: Using the Isolation Forest algorithm, detected a mojority of the poisoned data points. We then removed them from the dataset and achieved an MSE of 14.38.

4. **Implications**: Though Isolation Forest proved to be an effective technique to identify the poisoned data, it was not perfect. This shows that even with a very high success rate of identifying poisoned data, it can still negatively impact the model.

---